In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import os
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Flatten
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from PIL import ImageFile

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_dir = '/content/drive/MyDrive/Minor project/dataset/test'
valid_dir = '/content/drive/MyDrive/Minor project/dataset/val'
train_dir = '/content/drive/MyDrive/Minor project/dataset/train'
base_dir = '/content/drive/MyDrive/Minor project/dataset'

target_size = (400, 400)
batch_size = 32
ImageFile.LOAD_TRUNCATED_IMAGES = True

datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
width, height, dimension = 431, 467, 3
classes = os.listdir(train_dir)
classes.sort()

input_shape = tuple(list(target_size)+[3])
num_epochs = 10

print(classes)

['F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']


In [ ]:
input_layer = tf.keras.Input(shape=(width, height, 3), name='image_input')
resnet_model = Sequential()

pretrained_model= tf.keras.applications.resnet_v2.ResNet50V2(include_top=False,
                   input_shape=(width, height, 3),
                   pooling='max',
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

94683136/94668760 [==============================] - 0s 0us/step


In [ ]:
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(10, activation='softmax'))

In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24,619,018
Trainable params: 1,054,218
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
train_it = datagen.flow_from_directory(train_dir, class_mode='categorical', batch_size=batch_size
                                       , classes=classes, target_size=target_size)
test_it = datagen.flow_from_directory(test_dir, class_mode='categorical', batch_size=batch_size
                                       , classes=classes, target_size=target_size)
valid_it = datagen.flow_from_directory(valid_dir, class_mode='categorical', batch_size=batch_size
                                       , classes=classes, target_size=target_size)

train_sample = train_it.n
valid_sample = valid_it.n

print(train_it)
print(input_shape)
print(train_sample)
print(valid_sample)

Found 5009 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
(400, 400, 3)
5009
1000


In [ ]:
resnet_model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
history = resnet_model.fit(
        train_it, 
        steps_per_epoch=int(train_sample/batch_size),  
        epochs=num_epochs,
        validation_data=valid_it,
        validation_steps=int(valid_sample)/batch_size)

Epoch 1/10
156/156 [==============================] - 1349s 9s/step - loss: 2.3981 - accuracy: 0.8149 - val_loss: 0.4174 - val_accuracy: 0.9130
Epoch 2/10
156/156 [==============================] - 88s 562ms/step - loss: 0.5079 - accuracy: 0.9146 - val_loss: 0.8237 - val_accuracy: 0.8900
Epoch 3/10
156/156 [==============================] - 83s 528ms/step - loss: 0.3355 - accuracy: 0.9385 - val_loss: 0.7311 - val_accuracy: 0.8920
Epoch 4/10
156/156 [==============================] - 82s 525ms/step - loss: 0.3061 - accuracy: 0.9445 - val_loss: 0.3490 - val_accuracy: 0.9430
Epoch 5/10
156/156 [==============================] - 82s 525ms/step - loss: 0.2121 - accuracy: 0.9628 - val_loss: 0.4108 - val_accuracy: 0.9130
Epoch 6/10
156/156 [==============================] - 81s 517ms/step - loss: 0.1723 - accuracy: 0.9658 - val_loss: 0.3754 - val_accuracy: 0.9410
Epoch 7/10
156/156 [==============================] - 82s 523ms/step - loss: 0.1392 - accuracy: 0.9697 - val_loss: 0.2301 - val_acc

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()